In [1]:
# ==============================================================
# XGBoost Classification (XGB) Experiment
# ==============================================================
import pandas as pd
import numpy as np
import sys
import time
from datetime import datetime
import xgboost as xgb  # Make sure you have this installed: pip install xgboost

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from preprocessing import prepare_data

# Experiment config
random_state = 42
results = []
partial_save_path = "results/xgb_partial_results.csv"

In [2]:
# ---
# 1. Load Data
# ---
train_df = pd.read_csv("fraudTrain.csv")
test_df = pd.read_csv("fraudTest.csv")

print("Loaded train/test data.")

Loaded train/test data.


In [3]:
# ---
# 2. Prepare Data for TREE Models
# ---
# We MUST use mode="tree". This skips scaling and uses OrdinalEncoders.
out_train_init = prepare_data(
    train_df,
    mode="tree",
    training=False,  # We only need the encoders
    fit=True,
)
encoders = out_train_init["encoders"]
scalers = {}  # Scalers are not used

# Prepare TEST set using the *same* "tree" mode
out_test = prepare_data(
    test_df,
    mode="tree",
    training=False,
    fit=False,
    encoders=encoders,
    scalers=scalers,
)
df_test = out_test["df"]
X_test = df_test.drop("is_fraud", axis=1)
y_test = df_test["is_fraud"]

# Clean inf/-inf values
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0).clip(-1e6, 1e6)

print(f"Test set prepared for trees. Shape: {X_test.shape}")

Test set prepared for trees. Shape: (555719, 13)


In [4]:
# ---
# 3. Define Experiment Parameters
# ---
# Based on ALL previous results, we are only testing the best ratios.
ratios_to_test = [0.05, 0.1, 0.2] 
resample_types_to_test = ["df_up", "df_down"] 

# XGBoost specific parameters
# 'scale_pos_weight' is how XGBoost handles imbalance *internally*.
# We will test both our external resampling (what we've been doing)
# and its internal method.
params_to_test = [
    {
        "name": "XGB_Resampled", # Our method
        "params": {
            "n_estimators": 200, # More trees
            "max_depth": 10,       # Based on RF, deep trees are fine
            "learning_rate": 0.1,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "n_jobs": -1,
            "random_state": random_state,
            "scale_pos_weight": 1 # <-- We are handling balance with resampling
        },
    },
    {
        "name": "XGB_Depth_None", # Your new "unconstrained" experiment
        "params": {
            "n_estimators": 200,
            "max_depth": None,     # <-- No limit. This is your change.
            "learning_rate": 0.1,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "n_jobs": -1,
            "random_state": random_state,
            "scale_pos_weight": 1
        },
    }
]

In [7]:
# ---
# 4. Run Experiment Loop
# ---
print("\nStarting XGBoost experiment loop...")
for ratio_idx, ratio in enumerate(ratios_to_test, start=1):
    print(f"\n{'='*70}")
    print(
        f"[{datetime.now().strftime('%H:%M:%S')}] Starting ratio {ratio_idx}/{len(ratios_to_test)} → ratio={ratio}"
    )
    start_ratio_time = time.time()

    out_train = prepare_data(
        train_df,
        mode="tree",
        training=True,
        ratio=ratio,
        fit=False,  
        encoders=encoders,
        scalers=scalers,
    )

    for resample_type in resample_types_to_test:
        if resample_type not in out_train or out_train[resample_type] is None:
            continue

        df_train = out_train[resample_type]
        X_train = df_train.drop("is_fraud", axis=1)
        y_train = df_train["is_fraud"]

        X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0).clip(-1e6, 1e6)

        print(
            f"  [{datetime.now().strftime('%H:%M:%S')}] → Training on {resample_type} (samples={len(X_train):,})"
        )
        sys.stdout.flush()

        for p_info in params_to_test:
            model_name = p_info["name"]
            params = p_info["params"]
            start_k_time = time.time()

            print(f"    ⏳ Running {model_name} ...", end="")
            sys.stdout.flush()

            model = xgb.XGBClassifier(**params)
            
            model.fit(
                X_train, 
                y_train,
                eval_set=[(X_test, y_test)],
                verbose=False
            )

            # Predict
            y_pred = model.predict(X_test)
            y_prob = model.predict_proba(X_test)[:, 1]

            # Metrics
            f1 = f1_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            prec = precision_score(y_test, y_pred)
            acc = accuracy_score(y_test, y_pred)

            # === FIX #1: Handle AttributeError ===
            try:
                best_iter = model.best_iteration
            except AttributeError:
                best_iter = model.n_estimators # Set to full run if no early stop
            # =====================================

            results.append(
                {
                    "model": model_name,
                    "ratio": ratio,
                    "resample_type": resample_type.replace("df_",""),
                    "accuracy": acc,
                    "precision": prec,
                    "recall": rec,
                    "f1": f1,
                    "roc_auc": auc,
                    "best_iter": best_iter
                }
            )

            print(
                f" done → F1={f1:.4f}, Recall={rec:.4f}, AUC={auc:.4f} | Time={time.time() - start_k_time:.1f}s"
            )
            sys.stdout.flush()

    pd.DataFrame(results).to_csv(partial_save_path, index=False)
    print(f"  💾 Saved intermediate results → {partial_save_path}")
    print(f"  ✅ Completed ratio={ratio} in {(time.time() - start_ratio_time)/60:.1f} min")


Starting XGBoost experiment loop...

[23:51:22] Starting ratio 1/3 → ratio=0.05
  [23:51:26] → Training on df_up (samples=1,353,627)
    ⏳ Running XGB_Resampled ... done → F1=0.8475, Recall=0.7874, AUC=0.9969 | Time=30.4s
  [23:51:57] → Training on df_down (samples=157,626)
    ⏳ Running XGB_Resampled ... done → F1=0.7620, Recall=0.8573, AUC=0.9973 | Time=7.2s
  💾 Saved intermediate results → results/xgb_partial_results.csv
  ✅ Completed ratio=0.05 in 0.7 min

[23:52:04] Starting ratio 2/3 → ratio=0.1
  [23:52:07] → Training on df_up (samples=1,418,086)
    ⏳ Running XGB_Resampled ... done → F1=0.8513, Recall=0.8005, AUC=0.9968 | Time=33.7s
  [23:52:41] → Training on df_down (samples=82,566)
    ⏳ Running XGB_Resampled ... done → F1=0.6325, Recall=0.9002, AUC=0.9973 | Time=4.8s
  💾 Saved intermediate results → results/xgb_partial_results.csv
  ✅ Completed ratio=0.1 in 0.7 min

[23:52:46] Starting ratio 3/3 → ratio=0.2
  [23:52:50] → Training on df_up (samples=1,547,003)
    ⏳ Running 

In [8]:
# ---
# 5. Add the "Internal Resampling" Experiment (API FIX)
# ---
print(f"\n{'='*70}")
print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting Internal Imbalance Handling (scale_pos_weight)")
start_ratio_time = time.time()

out_train_orig = prepare_data(
    train_df,
    mode="tree",
    training=True,
    ratio=None, 
    fit=False,  
    encoders=encoders,
    scalers=scalers,
)

df_train_orig = out_train_orig["df_up"] 
X_train_orig = df_train_orig.drop("is_fraud", axis=1)
y_train_orig = df_train_orig["is_fraud"]

X_train_orig = X_train_orig.replace([np.inf, -np.inf], np.nan).fillna(0).clip(-1e6, 1e6)

n_legit = (y_train_orig == 0).sum()
n_fraud = (y_train_orig == 1).sum()
imbalance_ratio = n_legit / n_fraud
print(f"  Using original training data (samples={len(X_train_orig):,})")
print(f"  Calculated scale_pos_weight: {imbalance_ratio:.2f} (Legit: {n_legit}, Fraud: {n_fraud})")
sys.stdout.flush()

model_name = "XGB_Internal_ScalePos"
# === API FIX: MOVED PARAMS HERE ===
params = {
    "n_estimators": 200,
    "max_depth": 10,
    "learning_rate": 0.1,
    "objective": "binary:logistic",
    "n_jobs": -1,
    "random_state": random_state,
    "scale_pos_weight": imbalance_ratio,
    "early_stopping_rounds": 20,
    "eval_metric": "logloss"
}

start_k_time = time.time()
print(f"    ⏳ Running {model_name} ...", end="")
sys.stdout.flush()

model = xgb.XGBClassifier(**params)
# === API FIX: Removed early_stopping_rounds from .fit() ===
model.fit(
    X_train_orig, 
    y_train_orig,
    eval_set=[(X_test, y_test)],
    verbose=False
)

# Predict & Metrics
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
f1 = f1_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
prec = precision_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

results.append(
    {
        "model": model_name,
        "ratio": "internal", 
        "resample_type": "none",
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc,
        # === API FIX: Use .best_iteration attribute ===
        "best_iter": model.best_iteration
    }
)
print(
    f" done → F1={f1:.4f}, Recall={rec:.4f}, AUC={auc:.4f} | Time={time.time() - start_k_time:.1f}s"
)
print(f"  💾 Saved intermediate results → {partial_save_path}")
print(f"  ✅ Completed internal run in {(time.time() - start_ratio_time)/60:.1f} min")


[23:54:22] Starting Internal Imbalance Handling (scale_pos_weight)
  Using original training data (samples=1,296,675)
  Calculated scale_pos_weight: 171.75 (Legit: 1289169, Fraud: 7506)
    ⏳ Running XGB_Internal_ScalePos ... done → F1=0.8069, Recall=0.8065, AUC=0.9959 | Time=22.9s
  💾 Saved intermediate results → results/xgb_partial_results.csv
  ✅ Completed internal run in 0.4 min


In [9]:
# ---
# 6. Show Final Results
# ---
print("\nAll ratios completed.")
print(f"Total experiments logged: {len(results)}")
results_df = pd.DataFrame(results).sort_values(by="f1", ascending=False)
print("\n--- Top Performing XGBoost Models ---")
print(results_df.head(10))

results_df.to_csv("results/xgb_results.csv", index=False)
print("\nSaved final results to results/xgb_results.csv")


All ratios completed.
Total experiments logged: 7

--- Top Performing XGBoost Models ---
                   model     ratio resample_type  accuracy  precision  \
2          XGB_Resampled       0.1            up  0.998920   0.908947   
0          XGB_Resampled      0.05            up  0.998906   0.917436   
4          XGB_Resampled       0.2            up  0.998866   0.889460   
6  XGB_Internal_ScalePos  internal          none  0.998510   0.807280   
1          XGB_Resampled      0.05          down  0.997932   0.685682   
3          XGB_Resampled       0.1          down  0.995962   0.487503   
5          XGB_Resampled       0.2          down  0.992784   0.341223   

     recall        f1   roc_auc  best_iter  
2  0.800466  0.851264  0.996830        200  
0  0.787413  0.847466  0.996908        200  
4  0.806527  0.845966  0.997123        200  
6  0.806527  0.806903  0.995876        198  
1  0.857343  0.761964  0.997330        200  
3  0.900233  0.632493  0.997302        200  
5  0.93426

In [7]:
# ---
# 5. Run 'XGB_Depth_None' Experiment
# ---
# This cell ONLY tests the 'max_depth=None' (unconstrained) model
# to compare against the 'max_depth=10' models from the cell above.

print(f"\n{'='*70}")
print("Starting 'XGB_Depth_None' ONLY experiment loop...")
print(f"{'='*70}")

# Define ONLY the new parameters to test
params_to_test_new = [
    {
        "name": "XGB_Depth_None", # The "unconstrained" experiment
        "params": {
            "n_estimators": 200,
            "max_depth": None,     # <-- No limit
            "learning_rate": 0.1,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "n_jobs": -1,
            "random_state": random_state,
            "scale_pos_weight": 1
        },
    }
]

# This loop is identical to the one above, but uses 'params_to_test_new'
for ratio_idx, ratio in enumerate(ratios_to_test, start=1):
    print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Starting ratio {ratio_idx}/{len(ratios_to_test)} → ratio={ratio}")
    start_ratio_time = time.time()

    # Get the pre-processed data for this ratio
    out_train = prepare_data(
        train_df,
        mode="tree",
        training=True,
        ratio=ratio,
        fit=False,  
        encoders=encoders,
        scalers=scalers,
    )

    for resample_type in resample_types_to_test:
        if resample_type not in out_train or out_train[resample_type] is None:
            continue

        df_train = out_train[resample_type]
        X_train = df_train.drop("is_fraud", axis=1)
        y_train = df_train["is_fraud"]

        X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0).clip(-1e6, 1e6)

        print(
            f"  [{datetime.now().strftime('%H:%M:%S')}] → Training on {resample_type} (samples={len(X_train):,})"
        )
        sys.stdout.flush()

        # Iterate over our ONE new model config
        for p_info in params_to_test_new:
            model_name = p_info["name"]
            params = p_info["params"]
            start_k_time = time.time()

            print(f"    ⏳ Running {model_name} ...", end="")
            sys.stdout.flush()

            model = xgb.XGBClassifier(**params)
            
            model.fit(
                X_train, 
                y_train,
                eval_set=[(X_test, y_test)],
                verbose=False
            )

            # Predict
            y_pred = model.predict(X_test)
            y_prob = model.predict_proba(X_test)[:, 1]

            # Metrics
            f1 = f1_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            prec = precision_score(y_test, y_pred)
            acc = accuracy_score(y_test, y_pred)

            try:
                best_iter = model.best_iteration
            except AttributeError:
                best_iter = model.n_estimators

            # Append to the *SAME* results list
            results.append(
                {
                    "model": model_name,
                    "ratio": ratio,
                    "resample_type": resample_type.replace("df_",""),
                    "accuracy": acc,
                    "precision": prec,
                    "recall": rec,
                    "f1": f1,
                    "roc_auc": auc,
                    "best_iter": best_iter
                }
            )

            print(
                f" done → F1={f1:.4f}, Recall={rec:.4f}, AUC={auc:.4f} | Time={time.time() - start_k_time:.1f}s"
            )
            sys.stdout.flush()

    pd.DataFrame(results).to_csv(partial_save_path, index=False)
    print(f"  💾 Saved intermediate results → {partial_save_path}")
    print(f"  ✅ Completed ratio={ratio} in {(time.time() - start_ratio_time)/60:.1f} min")

print("\n✅ 'XGB_Depth_None' experiments are complete and added to results.")


Starting 'XGB_Depth_None' ONLY experiment loop...

[16:40:50] Starting ratio 1/3 → ratio=0.05
  [16:40:58] → Training on df_up (samples=1,353,627)
    ⏳ Running XGB_Depth_None ... done → F1=0.8164, Recall=0.8238, AUC=0.9973 | Time=43.2s
  [16:41:41] → Training on df_down (samples=157,626)
    ⏳ Running XGB_Depth_None ... done → F1=0.7670, Recall=0.8480, AUC=0.9975 | Time=17.2s
  💾 Saved intermediate results → results/xgb_partial_results.csv
  ✅ Completed ratio=0.05 in 1.1 min

[16:41:58] Starting ratio 2/3 → ratio=0.1
  [16:42:06] → Training on df_up (samples=1,418,086)
    ⏳ Running XGB_Depth_None ... done → F1=0.7630, Recall=0.8606, AUC=0.9976 | Time=29.7s
  [16:42:35] → Training on df_down (samples=82,566)
    ⏳ Running XGB_Depth_None ... done → F1=0.6487, Recall=0.8881, AUC=0.9973 | Time=4.0s
  💾 Saved intermediate results → results/xgb_partial_results.csv
  ✅ Completed ratio=0.1 in 0.7 min

[16:42:39] Starting ratio 3/3 → ratio=0.2
  [16:42:42] → Training on df_up (samples=1,547,

In [8]:
df_train = out_train_init["df"]

In [9]:
df_train

,merchant,category,amt,gender,state,city_pop,is_fraud,hour,day,month,weekday,is_weekend,age,distance_from_home
0,514,8,4.97,0,27,3495,0,0,1,1,1,0,31,78.597568
1,241,4,107.23,0,47,149,0,0,1,1,1,0,41,30.212176
2,390,0,220.11,1,13,4154,0,0,1,1,1,0,57,108.206083
3,360,2,45.00,1,26,1939,0,0,1,1,1,0,52,95.673231
4,297,9,41.96,1,45,99,0,0,1,1,1,0,33,77.556744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,499,0,15.56,1,44,258,0,12,21,6,6,1,59,119.752136
1296671,2,1,51.70,1,20,100,0,12,21,6,6,1,41,75.104085
1296672,599,1,105.93,1,32,899,0,12,21,6,6,1,53,99.047734
1296673,509,1,74.90,1,41,1126,0,12,21,6,6,1,40,84.627652


In [10]:
# For Amount
print(f"DEFAULT_AMT = {df_train['amt'].median():.2f}")

# For City Population
print(f"DEFAULT_CITY_POP = {df_train['city_pop'].median():.0f}")

# For Age
print(f"DEFAULT_AGE = {df_train['age'].median():.0f}")

# For Distance
print(f"DEFAULT_DISTANCE = {df_train['distance_from_home'].median():.2f}")

DEFAULT_AMT = 47.52
DEFAULT_CITY_POP = 2456
DEFAULT_AGE = 44
DEFAULT_DISTANCE = 78.23
